In [ ]:
# !pip install ydata-profiling
# !pip install ipywidgets
# !pip install seaborn
# !pip install hvplot
# !pip install ipympl
# !pip install plotly
# !pip install jupyter_bokeh
# ! pip install shap
# !pip install dash dash-bootstrap-components

In [ ]:
%cd ..

In [ ]:
data_path = "data/Quote_and_Order_Data_2025_01_30.csv"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import hvplot.pandas
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import shap

hvplot.extension('plotly')
output_notebook()
# %matplotlib widget
%matplotlib ipympl
%matplotlib inline
plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')
shap.initjs()

In [ ]:
data = pd.read_csv(
    data_path, 
    delimiter=",", 
    encoding="latin1", 
    # on_bad_lines="warn", 
    on_bad_lines='warn', 
    engine='python'
    )

# Data cleaning

In [ ]:
data = data[data["Seller Price"] > 0]

In [ ]:
data.groupby("Quote/Order Number").agg({"Quote/Order Status":"first"}).value_counts()

In [ ]:
data = data[data["Quote/Order Status"].isin(["Quote Returned", "Order Complete", "Order Waiting for Pickup or Delivery", "Planning Delivery"])]
# data = data[data["Quote/Order Status"].isin(["Quote Returned"])]

# Exploration

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
TOP_10_REGIONS = [
    "Florida",
    "Texas",
    "New York",
    "Georgia",
    "South Carolina",
    "California",
    "Tennessee",
    "Louisiana",
    "North Carolina",
    "Mississippi",
    "Maryland",
    "New Jersey",
    "Kentucky",
]

In [ ]:
CUSTOMER_COLUMN = "Buyer Name"
ORDER_DATE = "Quote/Order Created Date"
ORDER_ID_COLUMN = "Quote/Order Number"
PRODUCT_ID_COLUMN = "product_id"
PLANT_NAME_COLUMN = "Latin Name"
PLANT_CATEGORY_COLUMN = "Plant Category"
QUOTE_STATUS = "Quote/Order Status"
REGION_COLUMN = "Buyer Company Region"
SELLER_TOTAL = "Seller Total"
BUYER_TOTAL = "Buyer Total"
SELLER_PRICE = "Seller Price"
MARKUP_COLUMN = "Markup"

TOTAL_ITEMS_COLUMN = "Total Line Items"

REQUEST_PURPOSE_COLUMN = "Request Purpose"
QTO_COLUMN = "qto"

In [ ]:
data = data[data[REGION_COLUMN].isin(TOP_10_REGIONS)]

In [ ]:
data[TOTAL_ITEMS_COLUMN] = data["QTY"]
data[SELLER_TOTAL] = data["Seller Price"] * data["QTY"]
data[BUYER_TOTAL] = data["Buyer Price"] * data["QTY"]
data[MARKUP_COLUMN] = (data[BUYER_TOTAL] - data[SELLER_TOTAL])*100 / data[SELLER_TOTAL]
data[ORDER_DATE] = pd.to_datetime(data[ORDER_DATE], dayfirst=True).dt.date
data[QUOTE_STATUS] = data[QUOTE_STATUS].replace(
    {
        "Order Waiting for Pickup or Delivery":1,
        "Order Complete":1,
        "Planning Delivery":1,
        "Quote Returned":0,
    }
)

In [ ]:
data[ORDER_ID_COLUMN].nunique()

## Filter bad quotes

In [ ]:
lost_reasons_filepath = "data/Quote Lost Export-Feb132025.xlsx - All deals.csv"
lost_reasons = pd.read_csv(lost_reasons_filepath)
len(lost_reasons)

In [ ]:
lost_reasons["Quote Lost Reason"].isna().mean()

In [ ]:
lost_reasons[ORDER_ID_COLUMN] = lost_reasons["Deal Name"].apply(lambda x: "-".join(x.split("-")[:2]))
lost_reasons.head()

In [ ]:
lost_reasons["Quote Lost Reason"].value_counts()

In [ ]:
LOST_REASONS_TO_INCLUDE = [
    "Bad Fit Buyer - Not Competitive Pricing; Substitutions Not Approved",
    "Bad Fit Buyer - Not Competitive Pricing",
    "Good Fit Buyer - Not Competitive Pricing; Other",
    "Good Fit Buyer - Not Competitive Pricing; Substitutions Not Approved",
    "Good Fit Buyer - Not Competitive Pricing; Used their existing suppliers",
    "Good Fit Buyer - Not Competitive Pricing",
    "Other; Good Fit Buyer - Not Competitive Pricing",
    "Other; Used their existing suppliers",
    "Used their existing suppliers; Bad Fit Buyer - Not Competitive Pricing",
    "Used their existing suppliers; Good Fit Buyer - Not Competitive Pricing",
    "Used their existing suppliers; Other",
    "Used their existing suppliers",

    "Customer Lost Bid; Good Fit Buyer - Not Competitive Pricing",
    "Pictures not approved; Good Fit Buyer - Not Competitive Pricing",
    "Substitutions Not Approved; Bad Fit Buyer - Not Competitive Pricing",
]

In [ ]:
lost_reasons[lost_reasons["Quote Lost Reason"].isin(LOST_REASONS_TO_INCLUDE)]["Quote Lost Reason"].value_counts()

In [ ]:
lost_reasons[lost_reasons["Quote Lost Reason"].isin(LOST_REASONS_TO_INCLUDE)].shape

In [ ]:
x = set(data[ORDER_ID_COLUMN].unique()).intersection(lost_reasons[~lost_reasons["Quote Lost Reason"].isin(LOST_REASONS_TO_INCLUDE)][ORDER_ID_COLUMN].unique())
len(x)

In [ ]:
orders_to_remove = lost_reasons[~lost_reasons["Quote Lost Reason"].isin(LOST_REASONS_TO_INCLUDE)][ORDER_ID_COLUMN].to_list()
len(orders_to_remove)

In [ ]:
data = data[~data[ORDER_ID_COLUMN].isin(orders_to_remove)]
data[ORDER_ID_COLUMN].nunique()

# Explore distribution of quote status with respect to Request purpose / Type / QTO

In [ ]:
qto_and_request = data.groupby(ORDER_ID_COLUMN).agg({'Request Purpose':'first', 'qto':'first', 'Type':'first'})
qto_and_request["Request Purpose"].value_counts()

In [ ]:
qto_and_request["qto"].value_counts()

In [ ]:
def compute_quote_global_markup(rows):
    score = ((rows["Buyer Price"]*rows["QTY"]).sum() / (rows["Seller Price"]*rows["QTY"]).sum() - 1.0) * 100
    return score

AGG_MAPPING = {
        ORDER_DATE: "first",
        TOTAL_ITEMS_COLUMN: "sum",
        SELLER_TOTAL: "sum",
        
        QUOTE_STATUS: "first",
        
        REGION_COLUMN: "first",
        CUSTOMER_COLUMN: "first",
        
        REQUEST_PURPOSE_COLUMN: "first",
        QTO_COLUMN: "first",
    }

global_markups = data.groupby(ORDER_ID_COLUMN).apply(compute_quote_global_markup,include_groups=False)

per_quote_markup = data.groupby(ORDER_ID_COLUMN).agg(AGG_MAPPING)
per_quote_markup[MARKUP_COLUMN] = global_markups

In [ ]:
per_quote_markup.head()

In [ ]:
per_quote_markup[REGION_COLUMN].value_counts(normalize=True)

In [ ]:
per_quote_markup.groupby([QTO_COLUMN, REQUEST_PURPOSE_COLUMN])[QUOTE_STATUS].value_counts(sort=False).to_csv("rejections.csv")

In [ ]:
# per_quote_markup = per_quote_markup[per_quote_markup[REGION_COLUMN].isin(TOP_10_REGIONS)]

In [ ]:
# per_quote_markup[per_quote_markup[MARKUP_COLUMN].between(10,50)][QUOTE_STATUS].value_counts(normalize=True)
per_quote_markup[per_quote_markup[MARKUP_COLUMN] > 50][QUOTE_STATUS].value_counts(normalize=True)

# Visualization

In [ ]:
per_quote_markup[QTO_COLUMN] = per_quote_markup[QTO_COLUMN].replace(
    {
        "No":"Regular quote",
        "Yes": "Non regular quote"
    }
)

### success rate of quotes in qto vs non qto

In [ ]:
# success rate of quotes in qto vs non qto
import seaborn as sns
palette = sns.color_palette(palette='Paired')
sns.barplot(data=per_quote_markup[per_quote_markup[REGION_COLUMN].isin(TOP_10_REGIONS[:6])], x=REGION_COLUMN, y=QUOTE_STATUS, hue=QTO_COLUMN, errorbar=None, estimator="mean", palette=palette)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=18)
plt.xlabel("Region ", fontsize=18)
plt.ylabel("Conversion proportion", fontsize=18)
plt.show()

In [ ]:
# success rate of quotes in qto vs non qto
import seaborn as sns
palette = sns.color_palette(palette='Paired')
sns.barplot(data=per_quote_markup[per_quote_markup[REGION_COLUMN].isin(TOP_10_REGIONS[6:])], x=REGION_COLUMN, y=MARKUP_COLUMN, hue=QTO_COLUMN, errorbar=None, estimator="mean", palette=palette)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=18)
plt.xlabel("Regular vs Non regular quotes ", fontsize=18)
plt.ylabel("Average quote markup", fontsize=18)
plt.show()

### success rate of quotes in request purpose

In [ ]:
per_quote_markup[REQUEST_PURPOSE_COLUMN].value_counts()

In [ ]:
# success rate of quotes in bid vs install
import seaborn as sns
palette = sns.color_palette(palette='Paired')
sns.barplot(data=per_quote_markup[per_quote_markup[REGION_COLUMN].isin(TOP_10_REGIONS[6:])], x=REGION_COLUMN, y=QUOTE_STATUS, hue=REQUEST_PURPOSE_COLUMN, errorbar=None, estimator="mean", palette=palette)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=18)
plt.xlabel("Region ", fontsize=18)
plt.ylabel("Conversion proportion", fontsize=18)
plt.show()

In [ ]:
# success rate of quotes in qto vs non qto
import seaborn as sns
palette = sns.color_palette(palette='Paired')
sns.barplot(data=per_quote_markup[per_quote_markup[REGION_COLUMN].isin(TOP_10_REGIONS[6:])], x=REGION_COLUMN, y=MARKUP_COLUMN, hue=REQUEST_PURPOSE_COLUMN, errorbar=None, estimator="mean", palette=palette)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=18)
plt.xlabel("Regular vs Non regular quotes ", fontsize=18)
plt.ylabel("Average quote markup", fontsize=18)
plt.show()

# Build classifier

In [ ]:
data[ORDER_ID_COLUMN].nunique()

In [ ]:
SELECTED_COLUMNS = [
    CUSTOMER_COLUMN,
    ORDER_DATE,
    ORDER_ID_COLUMN,
    PRODUCT_ID_COLUMN,
    PLANT_CATEGORY_COLUMN,
    QUOTE_STATUS,
    REGION_COLUMN,
    SELLER_TOTAL,
    MARKUP_COLUMN,
    TOTAL_ITEMS_COLUMN,
    PLANT_NAME_COLUMN,
    REQUEST_PURPOSE_COLUMN,
    QTO_COLUMN,
    ]

In [ ]:
data = data[SELECTED_COLUMNS]

In [ ]:
data.head()

In [ ]:
original_data = data.copy()

In [ ]:
# data[PLANT_NAME_COLUMN].nunique()
# data[PLANT_NAME_COLUMN].value_counts().quantile(0.95)
data[PLANT_CATEGORY_COLUMN].value_counts()
# data[PLANT_CATEGORY_COLUMN].nunique()


In [ ]:
PLANT_CATEGORIES = data[PLANT_CATEGORY_COLUMN].unique()
PLANT_CATEGORIES

In [ ]:
PLANT_CATEGORY_INPUT = "Shrubs"

In [ ]:
data[QUOTE_STATUS].value_counts()

In [ ]:
# mask = (data[QTO_COLUMN] == "No") & (data[REQUEST_PURPOSE_COLUMN] == "For Bid")
# data = data[~mask]

high_markups_accepted_quotes = (data[MARKUP_COLUMN] >= 50.0) & (data[QUOTE_STATUS] == 1)
high_markups = data[high_markups_accepted_quotes].sample(frac=0.3, replace=False)

# high_markups = high_markups.sample(frac=1.0, replace=True)
a, b = 12.0, 30.0
high_markups[MARKUP_COLUMN] = (b - a) * np.random.random_sample(size=len(high_markups)) + a

In [ ]:
low_markups_refused_quotes = (data[MARKUP_COLUMN] <= 25.0) & (data[QUOTE_STATUS] == 0)
low_markups = data[low_markups_refused_quotes].sample(frac=0.5, replace=False)

a, b = 40.0, 80.0
low_markups[MARKUP_COLUMN] = (b - a) * np.random.random_sample(size=len(low_markups)) + a

In [ ]:
len(low_markups)

In [ ]:
# data = pd.concat([data, high_markups, low_markups])
# data[MARKUP_COLUMN] = data[MARKUP_COLUMN].clip(10.0, 40.0)

In [ ]:
# stop

In [ ]:
# train_data = data[data[PLANT_CATEGORY_COLUMN] == PLANT_CATEGORY_INPUT]
train_data = data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import (
    MinMaxScaler,
    OrdinalEncoder,
    RobustScaler,
    StandardScaler,
)
from sklearn.pipeline import FunctionTransformer, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, f1_score, PrecisionRecallDisplay, confusion_matrix, ConfusionMatrixDisplay 
import matplotlib.pyplot as plt
    

def _prepare_category_data(train_data):
    train_data = train_data[
    [
        QUOTE_STATUS,
        SELLER_TOTAL,
        MARKUP_COLUMN,
        TOTAL_ITEMS_COLUMN,
        REGION_COLUMN,
        PLANT_CATEGORY_COLUMN,
        PLANT_NAME_COLUMN,
        REQUEST_PURPOSE_COLUMN,
        QTO_COLUMN,
    ]
    ]
    train_data = train_data[train_data[REGION_COLUMN].isin(TOP_10_REGIONS)]
    print(train_data[QUOTE_STATUS].value_counts())

    accepted_quotes = train_data[train_data[QUOTE_STATUS] == 1]
    refused_quotes = train_data[train_data[QUOTE_STATUS] == 0].sample(int(len(accepted_quotes)*2))
    train_data = pd.concat([accepted_quotes, refused_quotes], ignore_index=True).sample(frac=1.0)
    return train_data


def train_model(train_data):
    train_data = _prepare_category_data(train_data)
    # Features and target
    X_train, X_test, y_train, y_test, pipeline = _fit_model(train_data)
    # Best threshold
    best_threshold = _retrieve_best_threshold(pipeline, X_test, y_test)
    pipeline.best_threshold = best_threshold
    # Display confusion matrix
    _display_cm(pipeline, X_test, y_test, best_threshold)

    return pipeline, X_train, X_test, y_train, y_test

def _fit_model(train_data):
    X = train_data.drop(QUOTE_STATUS, axis=1)
    y = train_data[QUOTE_STATUS]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

    # Column Transformer
    preprocessor = ColumnTransformer(
        transformers=[
            # ('markup', FunctionTransformer(func=lambda x:x), [MARKUP_COLUMN]),
            ('num', StandardScaler(), [TOTAL_ITEMS_COLUMN, SELLER_TOTAL, MARKUP_COLUMN]),
            ('cat_2', OrdinalEncoder(
                handle_unknown="use_encoded_value", 
                unknown_value=-1
                ),
                [
                    PLANT_CATEGORY_COLUMN, 
                    REGION_COLUMN,
                    REQUEST_PURPOSE_COLUMN,
                    QTO_COLUMN,
                    
                    ])
        ],
        # remainder="passthrough",
        remainder="drop",
    ).set_output(transform="pandas")

    # Pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        # ('classifier', GradientBoostingClassifier(random_state=42))
        ('classifier', HistGradientBoostingClassifier(
            random_state=42, 
            categorical_features=[
                                f"cat_2__{PLANT_CATEGORY_COLUMN}", 
                                f"cat_2__{REGION_COLUMN}", 
                                f"cat_2__{REQUEST_PURPOSE_COLUMN}", 
                                f"cat_2__{QTO_COLUMN}", 
                                  ]
                                  )
                                  ),
    ])
    
    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    
    return X_train,X_test,y_train,y_test,pipeline


def _retrieve_best_threshold(pipeline, X_test, y_test):
    
    display = PrecisionRecallDisplay.from_estimator(
        pipeline, X_test, y_test, name="Classifier", plot_chance_level=False
    )
    _ = display.ax_.set_title("Precision-Recall curve")

    # Predict probabilities on the test set
    y_prob = pipeline.predict_proba(X_test)[:, 1]

    # Get precision, recall, and thresholds
    precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
    f1_score = 2 * (precision * recall) / (precision + recall)
    best_f1_score_index = f1_score.argmax()
    best_threshold = thresholds[best_f1_score_index]
    print(best_threshold)
    return best_threshold


def _display_cm(pipeline, X_test, y_test, best_threshold):
    # Predictions 
    y_proba = pipeline.predict_proba(X_test) 
    y_pred = np.where(y_proba[:, 1] > best_threshold, 1, 0) 
    # Confusion Matrix s
    cm = confusion_matrix(y_test, y_pred) 
    # Display Confusion Matrix 
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["refused_quotes", "accepted quotes"]) 
    disp.plot(cmap=plt.cm.Blues) 
    plt.show()


In [ ]:
pipeline, X_train, X_test, y_train, y_test = train_model(train_data)

# Brier score
## Global score

In [ ]:
from sklearn.metrics import brier_score_loss, log_loss, roc_auc_score

# Predict probabilities on the test set
y_prob = pipeline.predict_proba(X_test)[:, 1]

# Compute Brier score
brier_score = brier_score_loss(y_test, y_prob)
entropy_score = log_loss(y_test, y_prob)
auc_score = roc_auc_score(y_test, y_prob)

print(f"Brier score: {brier_score}")
print(f"Entropy score: {entropy_score}")
print(f"AUC score: {auc_score}")

## Brier score for 3 most popular categories

In [ ]:
for category in [
    "Trees",
    "Shrubs",
    "Perennials",
    "Conifers",
    ]:
    selected_items = X_test[X_test[PLANT_CATEGORY_COLUMN] == category]
    selected_items_groundtruth = y_test.loc[selected_items.index]
    selected_items_proba = pipeline.predict_proba(selected_items)[:,1]
    score = brier_score_loss(selected_items_groundtruth, selected_items_proba)
    print(f"Brier score for category {category}: {score}")



# Sensitivity to markup

In [ ]:
X_train.head()

In [ ]:
preprocessor = pipeline.steps[0][1]
processed_X_train = preprocessor.transform(X_train)
processed_X_train.head()

In [ ]:
processed_X_train.dtypes

In [ ]:
pipeline.steps

In [ ]:
# model = pipeline.steps[1][1]
# df = processed_X_train.sample(2500)
# processed_X_test = preprocessor.transform(X_test).sample(2000)
# explainer = shap.explainers.Permutation(model.predict_proba, df)
# shap_values = explainer(processed_X_test)
# shap_values = shap_values[..., 1]

In [ ]:
# shap.plots.bar(shap_values)

In [ ]:
# shap.plots.waterfall(shap_values[0])

In [ ]:
# model.predict_proba(processed_X_test.iloc[:1])

# Partial dependency plot

In [ ]:
X_train.head()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
features = [1]
PartialDependenceDisplay.from_estimator(pipeline, X_test, features,
    kind='average')
plt.title("Partial dependence plot",  fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=18)
plt.xlabel("Markup value", fontsize=18)
plt.ylabel("Partial dependence", fontsize=18)

# Calibration Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import CalibrationDisplay
from matplotlib.ticker import PercentFormatter

fig = plt.figure(figsize=(10, 10))

# Create the first subplot for the calibration curve
ax_calibration_curve = fig.add_subplot(2, 1, 1)  # 2 rows, 1 column, 1st subplot

display = CalibrationDisplay.from_estimator(
    pipeline,
    X_test,
    y_test,
    n_bins=10,
    name="model",
    ax=ax_calibration_curve,
    strategy="uniform"
)
# Set the title of the calibration curve subplot
ax_calibration_curve.set_title("Calibration Curve", fontsize=20)

# # Set the font size for x and y ticks
ax_calibration_curve.set_xticklabels(ax_calibration_curve.get_xticks().round(1), fontsize=14)
ax_calibration_curve.set_yticklabels(ax_calibration_curve.get_yticks().round(1), fontsize=14)

# # Set the legend font size (assuming the legend already exists)
ax_calibration_curve.legend(fontsize=18)

# # Set the x and y labels with font size
ax_calibration_curve.set_xlabel("Mean predicted probability", fontsize=18)
ax_calibration_curve.set_ylabel("Fraction of positives", fontsize=18)


# Create the second subplot for the histogram
ax_hist = fig.add_subplot(2, 1, 2)  # 2 rows, 1 column, 2nd subplot

ax_hist.hist(
    display.y_prob,
    range=(0, 1),
    bins=10,
    label="model",
    weights=np.ones(len(display.y_prob)) / len(display.y_prob)
)
ax_hist.set_title("Histogram of Predicted Probabilities", fontsize=20)
ax_hist.set_xlabel("Predicted Probability", fontsize=18)
ax_hist.set_ylabel("Proportion", fontsize=18)
ax_hist.tick_params(axis='both', which='major', labelsize=14)
ax_hist.yaxis.set_major_formatter(PercentFormatter(1))

plt.tight_layout()
plt.show()
